In [2]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split



In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
df = pd.read_csv("/content/drive/MyDrive/augmented dataset/task1__eng_augmented.csv")
X_text = df['text'].tolist()
y_labels = df['polarization'].tolist()

In [10]:
print(df)

                                               id  \
0            eng_973938b90b0ff5d87d35a582f83f5c89   
1            eng_07dfd4600426caca6e2c5883fcbea9ea   
2            eng_f14519ff2302b6cd47712073f13bc461   
3            eng_e48b7e7542faafa544ac57b64bc80daf   
4            eng_7c581fb77bce8033aeba3d6dbd6273eb   
...                                           ...   
3343  eng_d118cc13746b8b3aa3066a621ccc13f9_aug667   
3344  eng_db607ce17c6337a53b97573b01100a12_aug668   
3345  eng_af2299a6d1a26c7bda07835fda65be6d_aug669   
3346  eng_67b71772937b16c9480050a767a28980_aug670   
3347  eng_d97075660229fab54952acf046dc7260_aug671   

                                                   text  polarization  
0              is defending imperialism in the dnd chat             0  
1     Still playing with this. I am now following Ra...             0  
2     .senate.gov Theres 3 groups out there Republic...             0  
3     "ABC MD, David Anderson, said the additional f...             0  
4  

In [6]:
model_name = "BAAI/bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
embedding_model = AutoModel.from_pretrained(model_name).to(device)
embedding_model.eval()

for p in embedding_model.parameters():
    p.requires_grad = False

def mean_pooling(model_output, attention_mask):
    token_embeds = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeds.size()).float()
    sum_embeddings = torch.sum(token_embeds * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [13]:
def get_all_embeddings(texts, model, tokenizer, device, batch_size=32):
    all_embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
        batch_texts = texts[i:i+batch_size]
        enc = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt').to(device)
        with torch.no_grad():
            outputs = model(**enc)
            embeddings = mean_pooling(outputs, enc['attention_mask'])
        all_embs.append(embeddings.cpu())
    return torch.cat(all_embs, dim=0)


X_embeddings = get_all_embeddings(X_text, embedding_model, tokenizer, device)
y_tensor = torch.tensor(y_labels, dtype=torch.long)




Embedding: 100%|██████████| 105/105 [00:30<00:00,  3.49it/s]


In [14]:
class Classifier(nn.Module):
    def __init__(self, embed_dim, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(embed_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        return self.model(x)


In [ ]:
# k = 5
# skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# X = X_embeddings.numpy()
# y = y_tensor.numpy()
# num_classes = len(np.unique(y))

# all_acc = []
# all_macro_f1 = []

# for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
#     print(f"\n=== Fold {fold+1} ===")

#     # Split
#     X_train, X_test = X[train_idx], X[test_idx]
#     y_train, y_test = y[train_idx], y[test_idx]

#     # Tensors
#     X_train_t = torch.tensor(X_train, dtype=torch.float32)
#     y_train_t = torch.tensor(y_train, dtype=torch.long)
#     X_test_t = torch.tensor(X_test, dtype=torch.float32)
#     y_test_t = torch.tensor(y_test, dtype=torch.long)

#     # DataLoader
#     train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=32, shuffle=True)

#     # Initialize classifier
#     classifier = Classifier(X_train_t.shape[1], num_classes).to(device)
#     optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)
#     criterion = nn.CrossEntropyLoss()

#     # Train
#     classifier.train()
#     for epoch in range(5):
#         for bx, by in train_loader:
#             bx, by = bx.to(device), by.to(device)
#             optimizer.zero_grad()
#             logits = classifier(bx)
#             loss = criterion(logits, by)
#             loss.backward()
#             optimizer.step()

#     # Evaluate
#     classifier.eval()
#     with torch.no_grad():
#         logits = classifier(X_test_t.to(device))
#         y_pred = torch.argmax(logits, dim=1).cpu().numpy()

#     acc = np.mean(y_pred == y_test)
#     macro_f1 = f1_score(y_test, y_pred, average='macro')

#     all_acc.append(acc)
#     all_macro_f1.append(macro_f1)

#     print(f"Fold {fold+1} - Accuracy: {acc:.4f} - Macro F1: {macro_f1:.4f}")

# # Cross-validation summary
# print(f"\n=== CV Summary ===")
# print(f"Average Accuracy: {np.mean(all_acc):.4f} ± {np.std(all_acc):.4f}")
# print(f"Average Macro F1: {np.mean(all_macro_f1):.4f} ± {np.std(all_macro_f1):.4f}")

In [15]:
X = X_embeddings.cpu().numpy()      # ensure numpy
y = y_tensor.cpu().numpy()

num_classes = len(np.unique(y))
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print("Train:", len(X_train), "Val:", len(X_val), "Test:", len(X_test))

# =========================================
# 3. Convert to tensors
# =========================================
X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_t = torch.tensor(y_train, dtype=torch.long).to(device)

X_val_t = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_t = torch.tensor(y_val, dtype=torch.long).to(device)

X_test_t = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_t = torch.tensor(y_test, dtype=torch.long).to(device)

train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=32, shuffle=True)

# =========================================
# 4. Initialize classifier
# =========================================
classifier = Classifier(X_train_t.shape[1], num_classes).to(device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# =========================================
# 5. Train classifier
# =========================================
classifier.train()
for epoch in range(30):
    for bx, by in train_loader:
        optimizer.zero_grad()
        logits = classifier(bx)
        loss = criterion(logits, by)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1} : {loss}")


Train: 2343 Val: 502 Test: 503
Epoch 1 : 0.2284710556268692
Epoch 2 : 0.4685346484184265
Epoch 3 : 0.06171118840575218
Epoch 4 : 0.594772458076477
Epoch 5 : 0.15651559829711914
Epoch 6 : 0.46882346272468567
Epoch 7 : 0.10536158084869385
Epoch 8 : 0.03735024482011795
Epoch 9 : 0.09163588285446167
Epoch 10 : 0.010717809200286865
Epoch 11 : 0.014162512496113777
Epoch 12 : 0.015078023076057434
Epoch 13 : 0.0006431358633562922
Epoch 14 : 0.011286874301731586
Epoch 15 : 0.0010956646874547005
Epoch 16 : 0.002343242522329092
Epoch 17 : 0.005821331404149532
Epoch 18 : 0.007098938338458538
Epoch 19 : 0.005451614502817392
Epoch 20 : 0.004309694282710552
Epoch 21 : 0.0038484057877212763
Epoch 22 : 0.005287600215524435
Epoch 23 : 0.0005875130300410092
Epoch 24 : 0.0053365896455943584
Epoch 25 : 0.028135571628808975
Epoch 26 : 0.0027001132257282734
Epoch 27 : 2.0775578377651982e-05
Epoch 28 : 0.009458056651055813
Epoch 29 : 0.0016769549110904336
Epoch 30 : 0.0009655405883677304


In [16]:
from sklearn.metrics import classification_report

classifier.eval()
with torch.no_grad():
    test_logits = classifier(X_test_t)
    test_preds = torch.argmax(test_logits, dim=1).cpu().numpy()

print("\n=== Classification Report ===")
print(classification_report(y_test, test_preds, digits=4))



=== Classification Report ===
              precision    recall  f1-score   support

           0     0.8095    0.8095    0.8095       252
           1     0.8088    0.8088    0.8088       251

    accuracy                         0.8091       503
   macro avg     0.8091    0.8091    0.8091       503
weighted avg     0.8091    0.8091    0.8091       503



In [18]:
# Suppose your classifier is trained and on device
model_path = "classifier_split_bge.pth"
torch.save(classifier.state_dict(), model_path)
print(f"Classifier saved to {model_path}")


Classifier saved to classifier_split_bge.pth


In [ ]:
#END